In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from streaming_llm.enable_streaming_llm import enable_streaming_llm
from streaming_llm.utils import load, download_url, load_jsonl
from examples.run_streaming_llama import streaming_inference
from tqdm import tqdm
import sys
import re
import time
import os
import json
import argparse
import torch
import warnings

In [ ]:
model_name_or_path = "lmsys/vicuna-13b-v1.3"
data_root = "data/"
model, tokenizer = load(model_name_or_path)
test_filepath = os.path.join(data_root, "mt_bench.jsonl")
print(f"Loading data from {test_filepath} ...")

if not os.path.exists(test_filepath):
    download_url(
        "https://raw.githubusercontent.com/lm-sys/FastChat/main/fastchat/llm_judge/data/mt_bench/question.jsonl",
        data_root,
    )
    os.rename(os.path.join(data_root, "question.jsonl"), test_filepath)

list_data = load_jsonl(test_filepath)
prompts = []
for sample in list_data:
    prompts += sample["turns"]

In [ ]:
kv_cache = enable_streaming_llm(
    model, start_size=4, recent_size=500, use_retrieval=True
)

In [ ]:
kv_cache.clear_cache()

In [ ]:
streaming_inference(
    model,
    tokenizer,
    prompts,
    kv_cache,
    max_gen_len=250
)